In [3]:
import os
import pandas as pd

ROOT_DIR   = "/NFS/celine/aicup/yoloNew"
KFOLDS     = 5
OUTPUT_PROJ= os.path.join(ROOT_DIR, "runs", "kfold_train")

records = []

for k in range(KFOLDS):
    #csv_path = os.path.join(OUTPUT_PROJ, f"fold{k}", "results.csv")
    csv_path = os.path.join(OUTPUT_PROJ, f"fold{k}2", "results.csv")
    if not os.path.exists(csv_path):
        print(f"[WARN] {csv_path} 不存在，跳過")
        continue

    df = pd.read_csv(csv_path)

    # 嘗試找 mAP@50 的欄位名稱（不同版本可能不一樣）
    cand_cols = ["metrics/mAP50", "metrics/mAP50(B)", "mAP50"]
    map_col = None
    for c in cand_cols:
        if c in df.columns:
            map_col = c
            break

    if map_col is None:
        raise KeyError(f"在 {csv_path} 找不到 mAP50 欄位，實際欄位有：{df.columns}")

    best_idx = df[map_col].idxmax()
    best_row = df.loc[best_idx]

    best_epoch = int(best_row["epoch"])
    best_map50 = float(best_row[map_col])

    records.append({
        "fold": k,
        "best_epoch": best_epoch,
        "best_mAP50": best_map50,
    })

summary = pd.DataFrame(records)
print(summary)
print("\n平均 mAP50:", summary["best_mAP50"].mean())
print("標準差:", summary["best_mAP50"].std())


   fold  best_epoch  best_mAP50
0     0         169     0.99032
1     1         170     0.98220
2     2         170     0.98591
3     3         165     0.98485
4     4         153     0.98634

平均 mAP50: 0.985924
標準差: 0.002937674250150963
